## Introduction

In this notebook we will propose you some example of how to run experiments with python-microscope.

These are just examples intended for training. In most cases you will want to use a full featured software package, 
with a dedicated GUI, like cockpit-microscope to run your experiments.

We are going to be running these example experiments using simulated devices. But the beauty of python-microscope 
is that the same code can be used to run experiments with real devices once they are connected to the PC. 

## Preparing the devices

Before we start, we need to prepare the devices. We will use a simulated camera, a simulated stage and a simulated
light source.

In [1]:
from microscope.simulators import SimulatedCamera, SimulatedStage, SimulatedLightSource
from microscope import AxisLimits

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# camera = SimulatedCamera()
camera = SimulatedCamera()
# Even a simulated stage needs to be initialized with the limits of the axes
stage = SimulatedStage(
    limits={'X' : AxisLimits(-5000, 5000),
            'Y' : AxisLimits(-10000, 12000),
            'Z' : AxisLimits(0, 1000)
            }
)
light_source = SimulatedLightSource()

## A simple z-stack

Let's start with a simple z-stack. Let's start to define some parameters for the experiment.

In [2]:
z_step_size = 0.35 # microns
# python-microscope stage do not use physical units. Instead, they use "counts". The number of counts per micron
# depends on the stage, the encoder it uses, the microstepping, etc. For a simulated stage, this does not really matter, 
# but for a real stage, you will need to know this number. You will find how to do so in the specific class implementation
z_counts_per_micron = 1
nr_of_z_steps = 20
exposure_time = 0.1 # seconds
light_power = 0.5 # as a fraction of the maximum power of the light source

Now we can start to run the experiment.

In [3]:
# Let's enable the camera so it gets ready to acquire images
camera.enable()

# let's set the camera exposure time
camera.set_exposure_time(exposure_time)

# A list to store the images
images = []

# Let's keep track of the current position, so we can come back to it once the experiment is done
pre_experiment_position = stage.position

# Let's move the stage to the starting position. 
# We will consider the current position to be the center of our stack, so we move half the number of steps
# We want to move in the negative direction, so we use a negative number of steps
half_z_stack = -nr_of_z_steps * z_step_size * z_counts_per_micron / 2
stage.move_by({"Z": half_z_stack})

# Let's loop over the z positions
for z_step in range(nr_of_z_steps):
    # Turn on the light
    light_source.enable()
    # Acquire an image and add it to the list
    images.append(camera.grab_next_data())
    # Turn off the light
    light_source.disable()
    # Move the stage now in the positive direction
    stage.move_by({"Z": z_step_size * z_counts_per_micron})
    
# Let's move the stage back to the original position
stage.move_to(pre_experiment_position)

Now all the images are in the buffer as 2D numpy arrays. We can start to get them out and putting them into a 3D numpy array.

In [4]:
# Let's create a 3D numpy array to store the images
image_3d = np.array([i[0] for i in images])

print(image_3d.shape)

(20, 512, 512)


and now some jupyter magic to display the 3D image

In [5]:
fig, ax = plt.subplots()
ax.set_xlim((0, image_3d.shape[1]))
ax.set_ylim((image_3d.shape[2],0))

im = ax.imshow(image_3d[0,:,:])
plt.close()

def init():
    im.set_data(image_3d[0,:,:])
    return (im,)
    
def animate(i):
    data_slice = image_3d[i,:,:]
    im.set_data(data_slice)
    return (im,)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=20, interval=200, blit=True)

HTML(anim.to_html5_video())